# Instrumental Investigation
The purpose of this file is to investigate the ability of different vad algorithms to reject instrumental sections.


In [3]:
import pandas as pd
import os
import sys
from collections import namedtuple

module_path = os.path.abspath(os.path.join('../vad_utils'))
if module_path not in sys.path:
    sys.path.append(module_path)
from vad_utils import SAMPLING_RATE, FRAME_SIZE_MS, SAMPLES_PER_FRAME
import vad_utils as vu
from pydub import AudioSegment
import matplotlib.pyplot as plt


In [2]:
fd = pd.read_csv("../../data/records_with_voxgrn_files.csv")
print(f'The columns of records are:\n{fd.columns}')
items = pd.read_csv("/prometheus/GRN/grid_program_items.csv")
print(f'Program items shape: {items.shape}')
# watch out for the missplelling. A t was actually dropped.
instrument = pd.read_csv("../../data/instrumenal_file_segs.csv")


The columns of records are:
Index(['Unnamed: 0.1', 'Unnamed: 0', 'iso', 'language_name', 'track',
       'location', 'year', 'path', 'filename', 'length', 'program', 'ID'],
      dtype='object')
Program items shape: (267681, 21)


Now how many segments were erroneously found using each sensitivity setting?

In [5]:
counts = instrument.sens.value_counts()
print(counts)

2    3679
1    2442
3    2402
0    2260
Name: sens, dtype: int64


OK - that was not expected. What happens if we reject the first segment for each file?

In [8]:
multi_seg_instrument = instrument[~instrument.file.duplicated(keep='first')]
multi_counts = multi_seg_instrument.sens.value_counts()
print(multi_counts)

0    971
2     55
3     22
1      4
Name: sens, dtype: int64


So the most robust rejection of instrumentals is to use sensitivity 1 and ignore the first segment. This is what I have in mind to do anyway as some files start with English.